# Processar extrações do [Data Feeds](https://experienceleague.adobe.com/en/docs/analytics/export/ftp-and-sftp/set-up-ftp-accounts/ftp-datafeeds)

Este notebook processa e filtra grandes volumes de dados de arquivos em formato TSV, extraídos do Data Feeds. O código lê os arquivos .tsv alocados em uma pasta de entrada, aplica filtragem com base em valores pré-definidos de uma coluna e salva os resultados em um arquivo CSV em uma pasta de saída. Ele também é otimizado para lidar com arquivos de grandes dimensões por meio de leitura em chunks.

In [1]:
import pandas as pd
from pathlib import Path

# Definindo a pasta base a partir da localização do notebook
base_dir = Path.cwd()  # Ou Path().resolve()
print(base_dir)

C:\Users\MarceloMarques\Desktop\pyautogui\Jupyter


In [2]:
# Definindo diretórios de entrada e saída relativos à pasta base
input_dir = base_dir / 'inputs'
output_dir = base_dir / 'outputs'
headers_file = input_dir / 'headers' / 'columns.csv'

# Certifica-se de que o diretório de saída existe
output_dir.mkdir(exist_ok=True)

# Definindo o arquivo de saída
output_file = output_dir / 'filtered_data.csv'

print(input_dir)
print(output_dir)
print(headers_file)
print(output_file)

C:\Users\MarceloMarques\Desktop\pyautogui\Jupyter\inputs
C:\Users\MarceloMarques\Desktop\pyautogui\Jupyter\outputs
C:\Users\MarceloMarques\Desktop\pyautogui\Jupyter\inputs\headers\columns.csv
C:\Users\MarceloMarques\Desktop\pyautogui\Jupyter\outputs\filtered_data.csv


⚠️ Certifique-se de que adicionou corretamente o arquivo com os cabeçalhos na pasta de entrada

In [3]:
# Lendo os cabeçalhos a partir do arquivo columns.csv, ignorando a primeira linha
headers_df = pd.read_csv(headers_file, skiprows=1, header=None)
# Converte o conteúdo da primeira coluna em uma lista
headers = headers_df[0].tolist() 

# Inicializando um DataFrame vazio para armazenar os dados concatenados
concatenated_df = pd.DataFrame(columns=headers)

headers

['date_time',
 'visit_num',
 'visit_page_num',
 'visit_start_pagename',
 'new_visit',
 'visid_timestamp',
 'visid_type',
 'post_visid_type',
 'visit_start_page_url',
 'visid_high',
 'visid_low',
 'mcvisid',
 'post_visid_high',
 'post_visid_low',
 'visid_new',
 'evar113',
 'post_evar113',
 'evar112',
 'post_evar112',
 'evar48',
 'post_evar48',
 'mobiledevice',
 'post_mobiledevice',
 'code_ver',
 'campaign',
 'post_campaign',
 'va_closer_id',
 'pagename',
 'evar1',
 'post_evar1',
 'last_hit_time_gmt']

### Edite manualmente o trecho abaixo 👇
Informe o nome da coluna que deseja filtrar, e os valores que serão filtrados

In [4]:
# Informe o nome da coluna a ser filtrada
filter_column = 'post_evar48'

# Informe os valores para filtrar na coluna. OBS: Os valores devem ser strings
filter_values = [
    "9660420",
    "35386133",
    "35399359",
    "36348153",
    "53002692", 
    "61551127",
    "83588016",
    "101994894",
    "104312857"
]
print(filter_column, filter_values)

post_evar48 ['9660420', '35386133', '35399359', '36348153', '53002692', '61551127', '83588016', '101994894', '104312857']


In [5]:
tsv_files = input_dir.glob('*.tsv')
tsv_files_list = list(tsv_files)  # Converte o iterador em uma lista
print(f"Número de arquivos .tsv encontrados: {len(tsv_files_list)}")

Número de arquivos .tsv encontrados: 29


In [6]:
# Definindo o tamanho dos chunks (ex: 100.000 linhas por vez)
chunk_size = 100000

# Iterando sobre todos os arquivos .tsv na pasta de entrada
for file_path in input_dir.glob('*.tsv'):
    print(f"\nProcessando arquivo: {file_path.name} ...")
    
    # Leitura do arquivo em chunks para evitar sobrecarga de memória. OBS: Todas as colunas são consideradas como de strings
    chunks = pd.read_csv(file_path, sep='\t', header=None, names=headers, chunksize=chunk_size, low_memory=True, dtype=str)

    # Processando cada chunk
    for i, chunk in enumerate(chunks):
        print(f"  ...chunk {i + 1}")  # Informação sobre o chunk atual
        
        # Filtrando as linhas onde a coluna de filtro contém os valores especificados
        filtered_chunk = chunk[chunk[filter_column].isin(filter_values)]
        
        # Exibindo quantos resultados foram encontrados no chunk atual
        if len(filtered_chunk) > 0:
            print(f"{len(filtered_chunk)} resultados encontrados")
        
        # Concatenando ao DataFrame principal
        concatenated_df = pd.concat([concatenated_df, filtered_chunk], ignore_index=True)
    
    print(f"Arquivo {file_path.name} processado com sucesso.")


Processando arquivo: 02-bancopansa.production_20240824-070000.tsv ...
  ...chunk 1
  ...chunk 2
  ...chunk 3
  ...chunk 4
  ...chunk 5
  ...chunk 6
  ...chunk 7
  ...chunk 8
  ...chunk 9
33 resultados encontrados
  ...chunk 10
  ...chunk 11
  ...chunk 12
Arquivo 02-bancopansa.production_20240824-070000.tsv processado com sucesso.

Processando arquivo: 02-bancopansa.production_20240824-080000.tsv ...
  ...chunk 1
55 resultados encontrados
  ...chunk 2
  ...chunk 3
  ...chunk 4
  ...chunk 5
  ...chunk 6
  ...chunk 7
  ...chunk 8
  ...chunk 9
  ...chunk 10
  ...chunk 11
  ...chunk 12
  ...chunk 13
  ...chunk 14
  ...chunk 15
  ...chunk 16
  ...chunk 17
  ...chunk 18
  ...chunk 19
  ...chunk 20
  ...chunk 21
  ...chunk 22
  ...chunk 23
  ...chunk 24
  ...chunk 25
  ...chunk 26
  ...chunk 27
46 resultados encontrados
  ...chunk 28
  ...chunk 29
  ...chunk 30
  ...chunk 31
  ...chunk 32
33 resultados encontrados
  ...chunk 33
  ...chunk 34
Arquivo 02-bancopansa.production_20240824-080000.ts

In [7]:
# Salvando o DataFrame concatenado em um arquivo .csv
concatenated_df.to_csv(output_file, index=False)

print(f"\nDados filtrados foram salvos em {output_file}.")


Dados filtrados foram salvos em C:\Users\MarceloMarques\Desktop\pyautogui\Jupyter\outputs\filtered_data.csv.
